Pre-Computation Setting

In [118]:
import numpy as np
import modern_robotics as mr
import csv

# Setting up for post-Computation:
M01 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.089159], [0, 0, 0, 1]]
M12 = [[0, 0, 1, 0.28], [0, 1, 0, 0.13585], [-1, 0, 0, 0], [0, 0, 0, 1]]
M23 = [[1, 0, 0, 0], [0, 1, 0, -0.1197], [0, 0, 1, 0.395], [0, 0, 0, 1]]
M34 = [[0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0.14225], [0, 0, 0, 1]]
M45 = [[1, 0, 0, 0], [0, 1, 0, 0.093], [0, 0, 1, 0], [0, 0, 0, 1]]
M56 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.09465], [0, 0, 0, 1]]
M67 = [[1, 0, 0, 0], [0, 0, 1, 0.0823], [0, -1, 0, 0], [0, 0, 0, 1]]
M07 = np.array(M01) @ np.array(M12) @ np.array(M23) @ np.array(M34) @ np.array(M45)\
      @ np.array(M56) @ np.array(M67)
G1 = np.diag([0.010267495893, 0.010267495893,  0.00666, 3.7, 3.7, 3.7])
G2 = np.diag([0.22689067591, 0.22689067591, 0.0151074, 8.393, 8.393, 8.393])
G3 = np.diag([0.049443313556, 0.049443313556, 0.004095, 2.275, 2.275, 2.275])
G4 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G5 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G6 = np.diag([0.0171364731454, 0.0171364731454, 0.033822, 0.1879, 0.1879, 0.1879])
Glist = [G1, G2, G3, G4, G5, G6]
Mlist = [M01, M12, M23, M34, M45, M56, M67] 
Slist = [[0,         0,         0,         0,        0,        0],
         [0,         1,         1,         1,        0,        1],
         [1,         0,         0,         0,       -1,        0],
         [0, -0.089159, -0.089159, -0.089159, -0.10915, 0.005491],
         [0,         0,         0,         0,  0.81725,        0],
         [0,         0,     0.425,   0.81725,        0,  0.81725]]
thetalist = np.array([0]*6)
dthetalist = np.array([0]*6)

Define the Puppet Function

In [149]:
# reference function which is part of the iteration
def referencePos(t,T_sb):
    if Problem_Number == 1 or Problem_Number == 2 or Problem_Number == 3:
        springPos_s = np.append(np.array([0,1,1]),[1])
        springPos_b = mr.TransInv(T_sb) @ springPos_s.T 
        springPos = np.array(springPos_b[0:3])

    else:
        y_position = np.sin(2*np.pi*t/5-np.pi/2)
        springPos_s = np.append(np.array([1,y_position,1]),[1])
        springPos_b = mr.TransInv(T_sb) @ springPos_s.T 
        springPos = np.array(springPos_b[0:3])

    return springPos

    

        


def Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,t,dt,damping=0,stiffness=0,restLength=0):
    #   Global Parameters
    N_step = int(t/dt)
    thetamat = np.array(thetalist).copy()
    dthetamat = np.array(dthetalist).copy()
    current_t = 0
    #   Start the iteration
    for i in range(N_step):
        # Position of end-effector in spatial frame
        T_sb = mr.FKinSpace(M07,Slist,thetalist)
        springPos = referencePos(current_t,T_sb) 
        # Procedure for spring force
        motion_vector = springPos 
        Length = np.linalg.norm(motion_vector)
        strain = Length - restLength
        Fspring_magnitude = stiffness * strain
        direction_vector = motion_vector / (Length + 1e-6)
        Force_tip = - direction_vector * Fspring_magnitude
        Ftip = np.hstack(([0, 0, 0],Force_tip))
        # taulist procedure if there's any non-zero damping 
        taulist = - dthetalist * damping
        # Call the mr.ForwardDynamics to calculate the acceleration
        ddthetalist = mr.ForwardDynamics(thetalist,dthetalist,taulist,g,Ftip,Mlist,Glist,Slist)
        # Euler approximation
        [thetalistNext,dthetalistNext] = mr.EulerStep(thetalist,dthetalist,ddthetalist,dt)
        # Substitude the position and velocity, get ready to the next iteration
        thetalist = thetalistNext
        dthetalist = dthetalistNext
        # Change the angle to the one in the acceptable range for CoppeliaSim 
        equivalent_thetalist = np.remainder(thetalist, 2 * np.pi)
        thetamat = np.vstack((thetamat,equivalent_thetalist))
        dthetamat = np.vstack((dthetamat,dthetalist))
        # iterate the current time, which is only useful for part 4a
        current_t += dt
    # csv file writting function
    with open(f'thetalist_matrix_{Problem_Number}{Sub_N}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(thetamat)

    with open(f'dthetalist_matrix_{Problem_Number}{Sub_N}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(dthetamat)

    return thetamat

Problem 1

In [122]:
Problem_Number = 1
Sub_N = 'a'

g = np.array([0, 0, -9.8])
stiffness = 0

Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,10,0.01,damping=0,stiffness=0,restLength=0)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.56975121e-03, 6.28031456e+00, 3.01000743e-04,
        6.28318531e+00, 6.28318531e+00],
       ...,
       [4.28941915e+00, 1.09537383e+00, 1.84475087e+00, 2.84129015e+00,
        4.36216093e+00, 5.57597437e+00],
       [4.33408265e+00, 9.85796372e-01, 2.05766672e+00, 2.74127587e+00,
        4.40163999e+00, 5.59507948e+00],
       [4.37245518e+00, 8.80989330e-01, 2.25186153e+00, 2.65514892e+00,
        4.43560326e+00, 5.61135777e+00]])

In [136]:
Sub_N = 'b'

g = np.array([0, 0, -9.8])
stiffness = 0

Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,10,0.05,damping=0,stiffness=0,restLength=0)

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.06424378, 6.21141651, 0.00752502, 6.28318531,
        6.28318531],
       ...,
       [5.60480562, 3.48447904, 4.92694187, 1.00524201, 3.82723998,
        6.02595256],
       [5.67972385, 4.19331149, 3.4382217 , 1.50196893, 3.69079866,
        6.18492027],
       [5.86093595, 4.57353177, 2.87948966, 1.38776325, 3.44470896,
        6.28203564]])

Problem 2

In [164]:
Problem_Number = 2

Sub_N = 'a'
damping = 3
stiffness = 0
g = np.array([0, 0, -9.8])

Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,5,0.01,damping,stiffness=0,restLength=0)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.56975121e-03, 6.28031456e+00, 3.01000743e-04,
        6.28318531e+00, 6.28318531e+00],
       ...,
       [1.28761269e-01, 1.48515074e+00, 6.26035109e+00, 6.03823432e+00,
        1.20318797e-02, 4.66892193e-03],
       [1.27771469e-01, 1.48175067e+00, 6.25699524e+00, 6.03656377e+00,
        1.17638751e-02, 4.59374137e-03],
       [1.26690184e-01, 1.47850131e+00, 6.25369877e+00, 6.03481936e+00,
        1.14881437e-02, 4.51314696e-03]])

In [141]:
Sub_N = 'b'
damping = -0.01
stiffness = 0
g = np.array([0, 0, -9.8])

Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,5,0.01,damping,stiffness=0,restLength=0)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.56975121e-03, 6.28031456e+00, 3.01000743e-04,
        6.28318531e+00, 6.28318531e+00],
       ...,
       [5.75277485e+00, 4.16741267e+00, 5.82274426e+00, 2.56063787e+00,
        5.69771314e+00, 1.76743122e+00],
       [5.74935553e+00, 4.13888442e+00, 5.91053630e+00, 2.50020829e+00,
        5.69403754e+00, 1.77912997e+00],
       [5.74581955e+00, 4.10836718e+00, 6.00054938e+00, 2.43956919e+00,
        5.69024372e+00, 1.79087257e+00]])

Problem 3(a)

In [163]:
Problem_Number = 3
Sub_N = 'a'

g = np.array([0]*3)
damping = 0
stiffness = 15
restLength = 0

Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,10,0.01,damping,stiffness,restLength)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [2.72588559e-04, 3.40254179e-05, 6.28211926e+00, 1.54757792e-03,
        6.28305950e+00, 6.28266975e+00],
       ...,
       [5.06585189e+00, 5.01172819e+00, 4.04838857e+00, 2.14443124e+00,
        4.59605273e+00, 2.03584258e+00],
       [5.10158908e+00, 4.99359865e+00, 4.04240055e+00, 2.22509402e+00,
        4.59326390e+00, 2.07875242e+00],
       [5.13710428e+00, 4.97514051e+00, 4.03802410e+00, 2.30504368e+00,
        4.59268223e+00, 2.11991324e+00]])

Problem 3(b)

In [147]:
Problem_Number = 3
Sub_N = 'b'

g = np.array([0]*3)
damping = 3
stiffness = 15
restLength = 0

Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,10,0.01,damping,stiffness,restLength)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [2.72588559e-04, 3.40254179e-05, 6.28211926e+00, 1.54757792e-03,
        6.28305950e+00, 6.28266975e+00],
       ...,
       [1.33583125e+00, 5.54210818e+00, 6.19112413e+00, 5.49800136e+00,
        8.71050067e-01, 1.00523672e-02],
       [1.33588371e+00, 5.54202008e+00, 6.19142600e+00, 5.49772152e+00,
        8.71499674e-01, 1.00490108e-02],
       [1.33593517e+00, 5.54193823e+00, 6.19173010e+00, 5.49744329e+00,
        8.71948676e-01, 1.00457083e-02]])

Problem 4(a)

In [150]:
Problem_Number = 4
Sub_N = 'a'

damping = 3
stiffness = 15
restLength = 0
g = np.array([0]*3)
Puppet(thetalist,dthetalist,g,Mlist,Slist,Glist,10,0.01,damping,stiffness,restLength)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [6.28282389e+00, 3.40254185e-05, 6.28211926e+00, 9.19336639e-04,
        6.28291297e+00, 1.12683371e-04],
       ...,
       [2.48918913e+00, 3.60653857e+00, 6.23959476e+00, 5.51372065e+00,
        6.71034992e-01, 6.42856908e-03],
       [2.48347300e+00, 3.60220519e+00, 6.23807399e+00, 5.51238564e+00,
        6.71238641e-01, 6.37651824e-03],
       [2.47786768e+00, 3.59792275e+00, 6.23663827e+00, 5.51106924e+00,
        6.71441513e-01, 6.32300215e-03]])